In [14]:
from filler import fill_redacted_text
import json
import tqdm
import time
import pandas as pd

In [2]:

data = pd.read_csv("inputs/radiology.csv")

print(data.head())



          note_id  subject_id     hadm_id note_type  note_seq  \
0  10000032-RR-14    10000032  22595853.0        RR        14   
1  10000032-RR-15    10000032  22595853.0        RR        15   
2  10000032-RR-16    10000032  22595853.0        RR        16   
3  10000032-RR-18    10000032         NaN        RR        18   
4  10000032-RR-20    10000032         NaN        RR        20   

             charttime            storetime  \
0  2180-05-06 21:19:00  2180-05-06 23:32:00   
1  2180-05-06 23:00:00  2180-05-06 23:26:00   
2  2180-05-07 09:55:00  2180-05-07 11:15:00   
3  2180-06-03 12:46:00  2180-06-03 14:01:00   
4  2180-07-08 13:18:00  2180-07-08 14:15:00   

                                                text  
0  EXAMINATION:  CHEST (PA AND LAT)\n\nINDICATION...  
1  EXAMINATION:  LIVER OR GALLBLADDER US (SINGLE ...  
2  INDICATION:  ___ HCV cirrhosis c/b ascites, hi...  
3  EXAMINATION:  Ultrasound-guided paracentesis.\...  
4  EXAMINATION:  Paracentesis\n\nINDICATION:  ___..

In [11]:
notes = data["text"]
notes.head()

0    EXAMINATION:  CHEST (PA AND LAT)\n\nINDICATION...
1    EXAMINATION:  LIVER OR GALLBLADDER US (SINGLE ...
2    INDICATION:  ___ HCV cirrhosis c/b ascites, hi...
3    EXAMINATION:  Ultrasound-guided paracentesis.\...
4    EXAMINATION:  Paracentesis\n\nINDICATION:  ___...
Name: text, dtype: object

In [12]:
notes = notes[:50]
notes

0     EXAMINATION:  CHEST (PA AND LAT)\n\nINDICATION...
1     EXAMINATION:  LIVER OR GALLBLADDER US (SINGLE ...
2     INDICATION:  ___ HCV cirrhosis c/b ascites, hi...
3     EXAMINATION:  Ultrasound-guided paracentesis.\...
4     EXAMINATION:  Paracentesis\n\nINDICATION:  ___...
5     EXAMINATION:  ULTRASOUND INTERVENTIONAL PROCED...
6     EXAMINATION:  LIVER OR GALLBLADDER US (SINGLE ...
7     EXAMINATION:  CHEST (PA AND LAT)\n\nINDICATION...
8     EXAMINATION:  Ultrasound-guided paracentesis.\...
9     EXAMINATION:  Ultrasound-guided paracentesis.\...
10    EXAMINATION:  ULTRASOUND PARACENTESIS\n\nINDIC...
11    EXAMINATION:  Ultrasound-guided paracentesis.\...
12    EXAMINATION:  PARACENTESIS\n\nINDICATION:  ___...
13    EXAMINATION:  THERAPEUTIC PARACENTESIS\n\nINDI...
14    EXAMINATION:  PARACENTESIS\n\nINDICATION:  ___...
15    INDICATION:  ___ year old woman with cirrhosis...
16    EXAMINATION:  PARACENTESIS\n\nINDICATION:  ___...
17    EXAMINATION:  CT HEAD W/O CONTRAST\n\nINDI

In [16]:

filled_data = []

for note in tqdm.tqdm(notes):
    retries = 0
    while retries < 3:
        try:
            filled_data.append(fill_redacted_text(note))
            break
        except Exception as e:
            retries += 1
            print(f"Error: {e}")
            time.sleep(1)

json.dump(filled_data, open("outputs/radiology_filled.json", "w"))

  0%|          | 0/50 [00:00<?, ?it/s]

 10%|█         | 5/50 [00:10<01:05,  1.46s/it]

Error: Invalid control character at: line 3 column 61 (char 63)
Error: Invalid control character at: line 3 column 61 (char 63)
Error: Invalid control character at: line 3 column 62 (char 64)


 54%|█████▍    | 27/50 [00:32<00:16,  1.37it/s]

Error: Invalid control character at: line 3 column 60 (char 62)
Error: Invalid control character at: line 3 column 59 (char 61)


...

 84%|████████▍ | 42/50 [00:48<00:05,  1.47it/s]

Error: Invalid control character at: line 3 column 52 (char 54)


 96%|█████████▌| 48/50 [00:53<00:01,  1.46it/s]

Error: Extra data: line 7 column 1 (char 2202)
Error: Extra data: line 8 column 1 (char 2203)
Error: Extra data: line 7 column 1 (char 2202)


100%|██████████| 50/50 [01:02<00:00,  1.25s/it]
